In [1]:
# import utility functions
import utils
from utils.flow_models import *
from utils.target import *
from utils.training import *

# import pytorch
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.distributions.transforms as transform
# set number of threads
torch.set_num_threads(8)

# numerical libs
import scipy
import scipy.io

# import plotting
import matplotlib.pyplot as plt
%matplotlib inline

# import I/O
import os
import sys

# set seed
seed = 14
np.random.seed(seed)
torch.manual_seed(0)

def loss_func_init(x, prior_logpdf, targ_logpdf):
        """ evaluate initial KL divergence between posterior distribution (NF + prior) 
        and target. x is samples without flow. This is a Monte-Carlo estimation of the 
        log partition function. """
        return (prior_logpdf - targ_logpdf(x)).mean()
    

## Rank 2 + ResNet

In [ ]:
# the wrapper is necessary, need to load mean and std data for each mixture Gaussian first
mixture_gauss_tf_dataset = utils.datasets.TensorizingFlowDataset("mixture_gaussian_patterned_truncated.mat", \
                                                              gaussian_data=False)
# input mean and std to wrapper
mu_input = np.array([mixture_gauss_tf_dataset.raw_dataset['mu1'][0], \
                     mixture_gauss_tf_dataset.raw_dataset['mu2'][0], \
                     mixture_gauss_tf_dataset.raw_dataset['mu3'][0], \
                     mixture_gauss_tf_dataset.raw_dataset['mu4'][0], \
                     mixture_gauss_tf_dataset.raw_dataset['mu5'][0]])
std_input = np.array([mixture_gauss_tf_dataset.raw_dataset['covmat1'], \
                      mixture_gauss_tf_dataset.raw_dataset['covmat2'], \
                      mixture_gauss_tf_dataset.raw_dataset['covmat3'], \
                      mixture_gauss_tf_dataset.raw_dataset['covmat4'], \
                      mixture_gauss_tf_dataset.raw_dataset['covmat5']])

def mixture_gaussian_logpdf_wrapper(theta, mu_list=mu_input, sig_list=std_input):
    return mixture_gaussian_logpdf(theta, mu_list, sig_list, scaling=2)


# initialize NF model
mixture_flow = NormalizingFlow(dim=mixture_gauss_tf_dataset.dim, blocks=RESNET_BLOCKS2_MIXTURE_GAUSSIAN, \
                            flow_length=5)

# begin training
mixture_report = train(mixture_gauss_tf_dataset, mixture_flow, mixture_gaussian_logpdf_wrapper, 
          num_epochs=100,
          batch_size=2**7,
          verbose=True,
          lr=5e-4,
          use_scheduler=True,
          grad_clip=1e+4)
    
scipy.io.savemat("./mixture_gaussian_tf_resnet_seed{}.mat".format(seed), {"report": mixture_report})

In [ ]:
X = mixture_report['post_training_samples'].copy()

In [ ]:
X.shape

In [ ]:
plt.scatter(X[:, 28], X[:, 29], s=1.2)

## Normalizing

In [ ]:
# the wrapper is necessary, need to load mean and std data for each mixture Gaussian first
mixture_gauss_nf_dataset = utils.datasets.TensorizingFlowDataset("mixture_gaussian_patterned_truncated.mat", \
                                                              gaussian_data=True)
# input mean and std to wrapper
mu_input = np.array([mixture_gauss_nf_dataset.raw_dataset['mu1'][0], \
                     mixture_gauss_nf_dataset.raw_dataset['mu2'][0], \
                     mixture_gauss_nf_dataset.raw_dataset['mu3'][0], \
                     mixture_gauss_nf_dataset.raw_dataset['mu4'][0], \
                     mixture_gauss_nf_dataset.raw_dataset['mu5'][0]])
std_input = np.array([mixture_gauss_nf_dataset.raw_dataset['covmat1'], \
                      mixture_gauss_nf_dataset.raw_dataset['covmat2'], \
                      mixture_gauss_nf_dataset.raw_dataset['covmat3'], \
                      mixture_gauss_nf_dataset.raw_dataset['covmat4'], \
                      mixture_gauss_nf_dataset.raw_dataset['covmat5']])

def mixture_gaussian_logpdf_wrapper(theta, mu_list=mu_input, sig_list=std_input):
    return mixture_gaussian_logpdf(theta, mu_list, sig_list, scaling=2)


# initialize NF model
mixture_flow = NormalizingFlow(dim=mixture_gauss_nf_dataset.dim, blocks=RESNET_BLOCKS2_MIXTURE_GAUSSIAN, \
                            flow_length=5)

# begin training
mixture_report2 = train(mixture_gauss_nf_dataset, mixture_flow, mixture_gaussian_logpdf_wrapper, 
          num_epochs=200,
          batch_size=2**7,
          verbose=True,
          lr=4e-4, 
          use_scheduler=True,
          grad_clip=1e+4)
    
scipy.io.savemat("./mixture_gaussian_nf_resnet_seed{}.mat".format(seed), {"report": mixture_report2})

## Visual Comparison